In [2]:
from concurrent.futures import ThreadPoolExecutor
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import json
import re

# Démarrage du chronomètre pour calculer la durée d'exécution du script
start_time = time.time()

# Importation du fichier CSV contenant les id
df_id = pd.read_csv('Data/df_for_scrap_tmdb.csv')

# Définition du nombre de requêtes effectuées en parallèle (Dépend du CPU)
nombre_req = 12

# Définition de la plage d'id à scraper
id_in = 0
id_out = 1
id = id_in

# Définition du chemin du fichier csv contenant les id
path_out = f'Data/tmdb_titres_originaux_{id_in}_{id_out}.csv'

# Définition de la variable initial pour pour suivre la progression
pourcent = 0

# Définition du HEADERS pour que les requetes ne soit pas bloquées
HEADERS = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.102 Safari/537.36 Edge/18.19582'}

# Définition de la fonction pour scraper les données de TMDB
def tmdb_titre_scrap(i):
    global pourcent, id
    id += 1

    # Initialisation des variables pour les informations à récupérer
    titre = budget = None
    
    try:
        # Construction de l'URL pour chaque film basé sur son ID
        url = f'https://www.themoviedb.org/movie/{i}'
        print(url)
        res = session.get(url, allow_redirects=True, headers=HEADERS)
        # Print du % avec une intervalle de 72 tests
        pourcent += 1
        if pourcent == 72:
            print('Execution :', round(((id-id_in)/(id_out-id_in))*100, 2), '%')
            pourcent = 0
        # Vérification du statut de la réponse, retourne des "0" si le code HTTP n'est pas 200 ou si l'URL a été redirigée
        if res.status_code != 200:
            return i, "0", "0"
        
        # Utilisation de BeautifulSoup pour parser le contenu HTML
        soup = BeautifulSoup(res.content, 'lxml')
        print(soup)
        # Extraction de tout le contenu entre des balises <p>
        p_tags = soup.find_all('p')

        # Cherchez la balise <p> contenant le mot "Budget"
        for p in p_tags:
            if 'Budget' in p.text:
                # Extrait le texte après "Budget"
                budget = p.text.split('Budget')[-1].strip()

        # Récupération du titre
        titre_temp = soup.find('strong', text='Original Title')
        if titre_temp:
            titre = titre_temp.next_sibling.strip()
        else:
            titre_temp_2 = soup.find('title')
            titre = titre_temp_2.get_text() if titre_temp_2 else ''
            titre = titre.split('(')[0].strip()

    # Gestion des exceptions liées aux requêtes HTTP
    except requests.exceptions.RequestException:
        return i, "0", "0"
    
    # Retourne les données extraites ou des "0" si les données ne sont pas disponibles
    resultat = (
        i,
        titre if titre else "0",
        budget if budget else "0",
    )
    return resultat

# Initialisation d'un DataFrame pour stocker les données extraites   
df = pd.DataFrame(columns=['id', 'titre', 'budget']) 
session = requests.Session()

# Utilisation d'un ThreadPoolExecutor pour exécuter les scrapings en parallèle et utiliser les differents coeurs logique du PC.
with ThreadPoolExecutor(max_workers=nombre_req) as executor:

    # Soumission des tâches de scraping pour chaque ID de film entre 1 et la variable total_film
    tests = [executor.submit(tmdb_titre_scrap, i) for i in df_id['id'][id_in:id_out]]
    for test in tests:
        i, titre, budget = test.result()
        variables = (titre, budget)
        if any(v != "0" for v in variables):
            df.loc[i] = [i, titre, budget]

# Arrêt du chronomètre et affichage de la durée d'exécution
end_time = time.time()
execution_time = end_time - start_time
print(f"Le script a pris {execution_time} secondes pour s'exécuter.")

# Sauvegarde des résultats dans un fichier CSV
df.to_csv(path_out, index=False)

# Affichage du DataFrame final
display(df)

https://www.themoviedb.org/movie/615656
Le script a pris 0.653850793838501 secondes pour s'exécuter.


,id,titre,budget
